In [1]:
# Setup

import pickle
import sklearn
from sklearn import cluster
import cv2
import numpy as np
import math
import time


# array processing functions

def getInterpolationDistribution(lenData, lenArray):
    interpCount = max(0, lenArray - lenData)
    # start with all the interpolations
    distribution = np.ones(lenData)
    i = 0
    currentSum = lenData
    while currentSum < lenArray:
        i=0
        while i < lenData and currentSum < lenArray:
            distribution[i] += 1
            currentSum += 1
            interpCount -=1
            i+=1            
    theSum = 0
    for v in distribution:
        theSum += v
    return distribution


def lerp(a, b, t):
    af = (float(a[0]), float(a[1]))
    bf = (float(b[0]), float(b[1]))
    x =  af[0] + t * (bf[0] - af[0]);
    y =  af[1] + t * (bf[1] - af[1]);
    return np.array([x,y])


def addInterpolations(inputData):
    processedArray = np.zeros([512, 2])

    interpDistribution = getInterpolationDistribution(len(inputData), 512)    
    iProcessed = 0
    iInput = 0
    div = 0
    
    for interp in interpDistribution:
        while interp > 0:
            if interp > 1:
                if interp > div:
                    div = float(interp)
                # interpolate
                a=0
                b=0
                if iInput==0:
                    a=inputData[0]
                    b=inputData[0]
                elif iInput >= len(inputData)-1:
                    a=inputData[len(inputData)-2]
                    b=inputData[len(inputData)-1]    
                else :
                    a = inputData[iInput - 1]
                    b = inputData[iInput]
                processedArray[iProcessed] = lerp(a,b, (div-interp)/div)
            else:
                # take next value from input
                processedArray[iProcessed] = inputData[iInput]
                iInput += 1
            interp -= 1
            iProcessed += 1

    return processedArray



def vectorise(src):
    # normalise the input image
    maxVal = 0.0
    for element in src:
        if abs(element[0]) > maxVal:
            maxVal = math.fabs(element[0])
        if abs(element[1]) > maxVal:
            maxVal = math.fabs(element[1])

    normalised = np.zeros([len(src),2])       
    for i, element in enumerate(src):
        normalised[i] = np.array([element[0] / maxVal,element[1] / maxVal])

    # calculate the vectors
    vectorised = np.zeros(len(src)*2)   

    for i, element in enumerate(normalised):
        if i < len(src)-1:
            vectorised[i*2] = normalised[i][0] - normalised[i+1][0]
            vectorised[(i*2) + 1] = normalised[i][1] - normalised[i+1][1]  
    return vectorised


# read the data
with open ('processedTrackPadDataFile', 'rb') as fp:
    itemlist = pickle.load(fp)

# cluster
kmeans = sklearn.cluster.KMeans(n_clusters=6)
kmeans_target = kmeans.fit_predict(itemlist)
          
print kmeans_target
print kmeans.cluster_centers_ 




[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0
 3 0 1 3 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3 3 4 4 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 4 5 5 5 4 5 4 4 4 5 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  3.32272565e-04
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.77680777e-04
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -2.58159618e-

In [2]:
# Set up clusters
clusterMapping = {0: "square", 3:"circle", 5:"heart", 1: "triangle", 2:"Z", 4: "lollypop"  }


# set up emojis
import json
from os import listdir, path

def winPath(path):
    p = path.replace('\\','/')
    p = p.replace('//','/')
    return p

def loadEmojis(folder):
    files = [f for f in listdir(folder) if path.isfile(path.join(folder, f))]
    emojis = {}
    names = []
    for file in files:
        name, ext = path.splitext(file)
        if ext == '.json':
            raw = json.load(open(folder + file, 'r'))
            name = raw.keys()[0]
            category = raw[name]
            emojis[name] = category[category.keys()[0]]
    return emojis
        

folder = winPath(r'D:\edu\phd\Computational Interaction\Group Project\EmojiCodeSheet\json\string\\')

emojis = loadEmojis(folder)            
    
    
def showEmojis(category):
    row1 = ''
    row2 = ''
    row3 = ''
    row4 = ''

    for cat in emojis:
        if cat == category:
            i=0
            for item in emojis[category]:
                it = item['value'] 
                if i%4== 0:
                    row1 = row1 + it
                elif i%4 == 1:
                    row2 = row2 + it
                elif i%4==2:
                    row3 = row3 + it
                else: 
                    row4 = row4 + it
                i = i+1

    print row1
    print row2
    print row3
    
showEmojis('natures')

🐶🐰🦁🐸🙈🐔🐣🐴🐌🦂🐠🐳🐅🐪🐏🐀🕊🐇🐲🌳☘🍃🌺🌼🌰🌎🌖🌒🌝🌙✨⛅🌧🔥☃🌪💧
🐱🐻🐮🐙🙉🐧🐥🦄🐞🦀🐟🐋🐃🐫🐑🐁🐕🐿🌵🌴🍀🍂🌻🌸🎃🌍🌗🌓🌛⭐☄🌥⛈💥⛄🌫💦
🐭🐼🐷🐵🙊🐦🐺🐝🐜🐍🐡🐊🐂🐘🐎🐓🐩🐾🎄🌱🎍🍁🌹💐🐚🌏🌘🌔🌜🌟☀🌦🌩❄🌬☂🌊


In [3]:
for cat in emojis:
    print cat
    
# gestureEmojiMapping = {"square": "objects", "circle": "natures", "heart" : "peoples", "triangle" : "symbols", "Z" : "activities", "lollypop" : "foods"  }
gestureEmojiMapping = {"square": "Flags", "circle": "Smileys", "heart" : "Hearts", "triangle" : "Symbols", "Z" : "Objects", "lollypop" : "Food"  }




activities
travels
natures
symbols
foods
objects
flags
peoples


In [6]:
import DrawingWindow
from os import listdir, path
import cv2

def winPath(path):
    p = path.replace('\\','/')
    p = p.replace('//','/')
    return p

def loadEmojis(folder):
    categFolders = [f for f in listdir(folder) if not path.isfile(path.join(folder, f))]
    emojiDict = dict()
    for categ in categFolders:
        catPath = path.join(folder, categ)
        emojis = [f for f in listdir(catPath) if path.isfile(path.join(catPath, f))]
        imgs = []
        for emoji in emojis:
            imgs.append(cv2.imread(catPath + '/' + emoji,cv2.IMREAD_UNCHANGED))
        emojiDict[categ] = imgs
    return emojiDict
            
emojiFolder = winPath(r'D:\edu\phd\Computational Interaction\Group Project\emojiPNG\\')
emojis = loadEmojis(emojiFolder)   


# test an input

w = DrawingWindow.DrawingWindow()
w.setup()

while(1):
    event = w.draw()
    if event == -1:
        break
    if w.draw() == 1:
        gestureIn = w.getLatestData()
        # w.close()
        yPos = 10
        xPos = 350
        w.clear()
        match = kmeans.predict(vectorise(addInterpolations(gestureIn)).reshape(1, -1))[0]
        for e in emojis[gestureEmojiMapping[clusterMapping[match]]]:
            w.displayImg(e, yPos, xPos)
            if (yPos + 40) < 460:
                yPos = yPos + 40
            else:
                yPos = 10
                xPos = xPos + 40
        print clusterMapping[match] + ": " + gestureEmojiMapping[clusterMapping[match]] 

        
        
# todo 
# display with transparency
# make clickable

# future:
# find distances
# rotate and mirror vectors for robustness to phase shift
# Supervised learning for cleanup 


Z: Objects
square: Flags
heart: Hearts
lollypop: Food
heart: Hearts


KeyboardInterrupt: 